In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installation

In [2]:
!pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.9/606.9 kB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 56.

In [3]:
!pip install python-dotenv

In [4]:
import langchain
import openai
import tiktoken
import deeplake
import os
import dotenv
from dotenv.main import load_dotenv

In order to access OpenAI's services, you must first obtain credentials by signing up on their website, completing the registration process, and creating an API key from your dashboard. This enables you to leverage OpenAI's powerful capabilities in your projects.

1. If you don't have an account yet, create one by going to https://platform.openai.com/. If you already have an account, skip to step 5.

2. Fill out the registration form with your name, email address, and desired password.

3. OpenAI will send you a confirmation email with a link. Click on the link to confirm your account.
4. Please note that you'll need to verify your email account and provide a phone number for verification.
5. Log in to https://platform.openai.com/.
6. Navigate to the API key section at https://platform.openai.com/account/api-keys.
7. Click "Create new secret key" and give the key a recognizable name or ID.

**The LLMs** \

The fundamental component of LangChain involves invoking an LLM with a specific input. To illustrate this, we'll explore a simple example. Let's imagine we are building a service that suggests personalized workout routines based on an individual's fitness goals and preferences.

To accomplish this, we will first need to import the LLM wrapper.

In [5]:
from langchain.llms import OpenAI

The temperature parameter in OpenAI models manages the randomness of the output. When set to 0, the output is mostly predetermined and suitable for tasks requiring stability and the most probable result. At a setting of 1.0, the output can be inconsistent and interesting but isn't generally advised for most tasks. For creative tasks, a temperature between 0.70 and 0.90 offers a balance of reliability and creativity. The best setting should be determined by experimenting with different values for each specific use case. The code initializes the GPT-3.5 model’s Turbo variant. We will learn more about the various models and their differences later on.

Before executing the following code, save your OpenAI key in the environment variable using the following key OPENAI_API_KEY

In [6]:
# with open('/content/drive/MyDrive/openai_api_key.txt','r') as file:
#     api = " ".join(line.rstrip() for line in file)
# open_ai_api_key=api.split(":")[1]

In [17]:
load_dotenv('/content/drive/MyDrive/.env')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [8]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

In [34]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))



Day 1:
Warm-up:
- 10 minute jog around the neighborhood
- 5 minute dynamic stretching (lunges, leg swings, arm circles)

Workout:
- 30 minutes of cycling on a hilly route
- 20 minute jog on a trail or in a park
- 5 minute cool down walk

Day 2:
Warm-up:
- 5 minute brisk walk
- 5 minute dynamic stretching (squats, arm swings, high knees)

Workout:
- 20 minute HIIT (high-intensity interval training) workout using body weight exercises (burpees, jumping jacks, mountain climbers)
- 10 minute walk on a steep incline
- 20 minute bike ride at a moderate pace
- 5 minute cool down walk

Day 3:
Warm-up:
- 10 minute jog on a track or in a park
- 5 minute dynamic stretching (leg kicks, trunk rotations, arm circles)

Workout:
- 45 minute hike on a challenging trail
- 10 minute jog on a flat surface
- 5 minute cool down walk

Day 4:
Warm-up:
- 5 minute walk
- 5 minute dynamic stretching (lunge twists, arm circles, leg


**The Chains**

In LangChain, a chain is an end-to-end wrapper around multiple individual components, providing a way to accomplish a common use case by combining these components in a specific sequence. The most commonly used type of chain is the LLMChain, which consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser.

The LLMChain works as follows:

1. Takes (multiple) input variables.
2. Uses the PromptTemplate to format the input variables into a prompt.
3. Passes the formatted prompt to the model (LLM or ChatModel).
4. If an output parser is provided, it uses the OutputParser to parse the output of the LLM into a final format

In the next example, we demonstrate how to create a chain that generates a possible name for a company that produces eco-friendly water bottles. By using LangChain's LLMChain, PromptTemplate, and OpenAIclasses, we can easily define our prompt, set the input variables, and generate creative outputs.


In [9]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))



1. PureLife Bottles
2. EarthSip
3. GreenWave Bottles
4. EcoHydrate
5. Sustainable Sippers
6. NatureNurture Bottles
7. ReNew Bottles
8. EcoAqua
9. PureEarth Bottles
10. GreenHue Hydration


**The Memory**

In LangChain, Memory refers to the mechanism that stores and manages the conversation history between a user and the AI. It helps maintain context and coherency throughout the interaction, enabling the AI to generate more relevant and accurate responses. Memory, such as ConversationBufferMemory, acts as a wrapper around ChatMessageHistory, extracting the messages and providing them to the chain for better context-aware generation.

In [9]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Well, I am an artificial intelligence created by a team of programmers and engineers. My purpose is to assist and interact with humans in various tasks and conversations. I am constantly learning and improving through algorithms and data analysis. I am currently housed in a server room with other

**Deep Lake VectorStore**

Deep Lake provides storage for embeddings and their corresponding metadata in the context of LLM apps. It enables hybrid searches on these embeddings and their attributes for efficient data retrieval. It also integrates with LangChain, facilitating the development and deployment of applications.

Deep Lake provides several advantages over the typical vector store:

1. It’s multimodal, which means that it can be used to store items of diverse modalities, such as texts, images, audio, and video, along with their vector representations.
2. It’s serverless, which means that we can create and manage cloud datasets without creating and managing a database instance. This aspect gives a great speedup to new projects.
3. Last, it’s possible to easily create a data loader out of the data loaded into a Deep Lake dataset. It is convenient for fine-tuning machine learning models using common frameworks like PyTorch and TensorFlow.

In order to use Deep Lake, you first have to register on the Activeloop website and redeem your API token. Here are the steps for doing it:

1. Sign up for an account on Activeloop's platform. You can sign up at Activeloop's website. After specifying your username, click on the “Sign up” button. You should now see your homepage.
2. You should now see a “Create API token” button at the top of your homepage. Click on it, and you’ll get redirected to the “API tokens” page. This is where you can generate, manage, and revoke your API keys for accessing Deep Lake.
3. Click on the "Create API token" button. Then, you should see a popup asking for a token name and an expiration date. By default, the token expiration date is set so that the token expires after one day from its creation, but you can set it further in the future if you want to keep using the same token for the whole duration of the course. Once you’ve set the token name and its expiration date, click on the “Create API token” button.
4. You should now see a green banner saying that the token has been successfully generated, along with your new API token, on the “API tokens” page. To copy your token to your clipboard, click on the square icon on its right.

In [18]:
ACTIVELOOP_TOKEN = os.getenv('ACTIVELOOP_TOKEN')

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [10]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "parthaai"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:08<00:00,  8.79s/it]

Dataset(path='hub://parthaai/langchain_course_from_zero_to_hero', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   


['1315da00-387e-11ef-bd20-0242ac1c000c',
 '1315db90-387e-11ef-bd20-0242ac1c000c']

Now let's create a retrivalQA chain

In [11]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

Next, let's create an agent that uses the RetrievalQA chain as a tool

In [12]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [13]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I should use the Retrieval QA System to answer this question
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation: 
Napoleon Bonaparte was born in 15 August 1769.
Thought: I now know the final answer
Final Answer: Napoleon Bonaparte was born in 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born in 15 August 1769.


This example demonstrates how to use Deep Lake as a vector database and create an agent with a RetrievalQA chain as a tool to answer questions based on the given document.
Let’s add an example of reloading an existing vector store and adding more data.

We first reload an existing vector store from Deep Lake that's located at a specified dataset path. Then, we load new textual data and split it into manageable chunks. Finally, we add these chunks to the existing dataset, creating and storing corresponding embeddings for each added text segment:

In [14]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://parthaai/langchain_course_from_zero_to_hero already exists, loading from the storage


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:12<00:00, 12.52s/it]

Dataset(path='hub://parthaai/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['aeed874c-387f-11ef-bd20-0242ac1c000c',
 'aeed88fa-387f-11ef-bd20-0242ac1c000c']

We then recreate our previous agent and ask a question that can be answered only by the last documents added.

In [15]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [16]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
 I should use the Retrieval QA System to find the answer.
Action: Retrieval QA System
Action Input: "When was Michael Jordan born?"
Observation:  Michael Jordan was born on 17 February 1963.
Thought: I now know the final answer.
Final Answer: Michael Jordan was born on 17 February 1963.

> Finished chain.
Michael Jordan was born on 17 February 1963.


The LLM successfully retrieves accurate information by using the power of Deep Lake as a vector store and the OpenAI language model.

**Agents in LangChain**

In LangChain, agents are high-level components that use language models (LLMs) to determine which actions to take and in what order. An action can either be using a tool and observing its output or returning it to the user. Tools are functions that perform specific duties, such as Google Search, database lookups, or Python REPL.

Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.

Several types of agents are available in LangChain:

1. The *zero-shot-react-description* agent uses the ReAct framework to decide which tool to employ based purely on the tool's description. It necessitates a description of each tool.
2. The *react-docstore agent* engages with a docstore through the ReAct framework. It needs two tools: a Search tool and a Lookup tool. The Search tool finds a document, and the Lookup tool searches for a term in the most recently discovered document.
3. The *self-ask-with-search agent* employs a single tool named Intermediate Answer, which is capable of looking up factual responses to queries. It is identical to the original self-ask with the search paper, where a Google search API was provided as the tool.
4. The *conversational-react-description agent* is designed for conversational situations. It uses the ReAct framework to select a tool and uses memory to remember past conversation interactions.

In our example, the Agent will use the Google Search tool to look up recent information about the Mars rover and generates a response based on this information.

First, you want to set the environment variables “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use Google Search via API. Refer to this article for a guide on how to get them.

Google Search

In [19]:
%pip install --upgrade --quiet  langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 18.9 MB/s eta 0:00:00


In [20]:
GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [28]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [29]:
tool.run("Obama's first name?")

"1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb ... OBAMA, II. Day. 4. 6b. Island. Year. 5b. Hour. 1961 7:24 P.M.. Oahu. 6d. Is Place\xa0... As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing\xa0... Apr 2, 2018 ... His full name is Barack Hussein Obama. Please note the middle name is Arabic. In the Islam faith the son is automatically the religion of his\xa0... Apr 7, 2021 ... Morons, Obama IS his last name, and his first name is President. Reply\xa0... A museum that asks you to believe—not just in President Obama's power to create change, but in your own. ... (All fields required.) First Name. First Name. Last\xa0... Barack Hussein Obama II was born August 4, 1961, in Honolulu, Hawaii, to parents Barack H. Obama, Sr., and Stanley Ann Dunham. His parents divorced when he\xa0... Follow the Girls Opportunity Alliance to help adolescent girls and the grassroot

Number of Results

You can use the k parameter to set the number of results

In [30]:
search = GoogleSearchAPIWrapper(k=1)

tool = Tool(
    name="I'm Feeling Lucky",
    description="Search Google and return the first result.",
    func=search.run,
)

In [31]:
tool.run("python")

'The official home of the Python Programming Language.'

Then, let’s import the necessary modules:

1. langchain.llms.OpenAI: This is used to create an instance of the OpenAI language model, which can generate human-like text based on the input it's given.
2. langchain.agents.load_tools: This function is used to load a list of tools that an AI agent can use.
3. langchain.agents.initialize_agent: This function initializes an AI agent that can use a given set of tools and a language model to interact with users.
4. langchain.agents.Tool: This is a class used to define a tool that an AI agent can use. A tool is defined by its name, a function that performs the tool's action, and a description of the tool.
5. langchain.utilities.GoogleSearchAPIWrapper: This class is a wrapper for the Google Search API, allowing it to be used as a tool by an AI agent. It likely contains a method that sends a search query to Google and retrieves the results.

In [32]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [33]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

In [34]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

The Tool object represents a specific capability or function the system can use. In this case, it's a tool for performing Google searches.

It is initialized with three parameters:

1. name parameter: This is a string that serves as a unique identifier for the tool. In this case, the name of the tool is "google-search.”
2. func parameter: This parameter is assigned the function that the tool will execute when called. In this case, it's the run method of the search object, which presumably performs a Google search.
3. description parameter: This is a string that briefly explains what the tool does. The description explains that this tool is helpful when you need to use Google to answer questions about current events.

In [35]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

Next, we create an agent that uses our Google Search tool:

1. initialize_agent(): This function call creates and initializes an agent. An agent is a component that determines which actions to take based on user input. These actions can be using a tool, returning a response to the user, or something else.
2. tools:  represents the list of Tool objects that the agent can use.
3. agent="zero-shot-react-description": The "zero-shot-react-description" type of an Agent uses the ReAct framework to decide which tool to use based only on the tool's description.
3. verbose=True: when set to True, it will cause the Agent to print more detailed information about what it's doing. This is useful for debugging and understanding what's happening under the hood.
4. max_iterations=6: sets a limit on the number of iterations the Agent can perform before stopping. It's a way of preventing the agent from running indefinitely in some cases, which may have unwanted monetary costs.

In [36]:
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=6)

In [37]:
response = agent("What's the latest news about the Mars rover?")
print(response['output'])



> Entering new  chain...
 I should search for news articles about the Mars rover.
Action: google-search
Action Input: "Mars rover news"
Observation: Mars rovers can only make exciting new discoveries thanks to human scientists making careful decisions about their next stop. The Mars 2020 mission is aimed ... NASA Mars rover, exploring since February 2021. Hobbies: Photography, collecting rocks, off-roading. Team HQ @NASAJPL (Verification: ... Mars rovers can only make exciting new discoveries thanks to human scientists making careful decisions about their next stop. The Mars 2020 mission is aimed ... Oct 19, 2022 ... ... Curiosity, visit: http://mars.nasa.gov/msl. Get the Latest JPL News. SUBSCRIBE TO THE NEWSLETTER. News Media Contact. Andrew Good. Jet ... Aug 6, 2012 ... NASA's Curiosity Mars rover appears as a dark speck in this image captured from directly overhead ... News and Features. All Curiosity Rover News. Dec 13, 2023 ... Nasa says its Perseverance rover has essentially c

In summary, Agents in LangChain help decide which actions to take based on user input. The example demonstrates initializing and using a "zero-shot-react-description" agent with a Google search tool.

**Tools in LangChain**

LangChain provides a variety of tools for agents to interact with the outside world. These tools can be used to create custom agents that perform various tasks, such as searching the web, answering questions, or running Python code. In this section, we will discuss the different tool types available in LangChain and provide examples of creating and using them.

In our example, two tools are being defined for use within a LangChain agent: a Google Search tool and a Language Model tool acting specifically as a text summarizer. The Google Search tool, using the GoogleSearchAPIWrapper, will handle queries that involve finding recent event information. The Language Model tool leverages the capabilities of a language model to summarize texts. These tools are designed to be used interchangeably by the agent, depending on the nature of the user's query.

Let’s import the necessary libraries.

In [38]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

We then instantiate a  LLMChain specifically for text summarization.

In [39]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

Next, we create the tools that our agent will use

In [40]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

We are now ready to create our agent that leverages two tools.

In [41]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

Let’s run the agent with a question about summarizing the latest news about the Mars rover.

In [42]:
response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])



> Entering new  chain...
 I should use the Search tool to find recent news about the Mars rover.
Action: Search
Action Input: "Mars rover news"
Observation: Mars rovers can only make exciting new discoveries thanks to human scientists making careful decisions about their next stop. The Mars 2020 mission is aimed ... NASA Mars rover, exploring since February 2021. Hobbies: Photography, collecting rocks, off-roading. Team HQ @NASAJPL (Verification: ... Mars rovers can only make exciting new discoveries thanks to human scientists making careful decisions about their next stop. The Mars 2020 mission is aimed ... Oct 19, 2022 ... ... Curiosity, visit: http://mars.nasa.gov/msl. Get the Latest JPL News. SUBSCRIBE TO THE NEWSLETTER. News Media Contact. Andrew Good. Jet ... Aug 6, 2012 ... NASA's Curiosity Mars rover appears as a dark speck in this image captured from directly overhead ... News and Features. All Curiosity Rover News. Dec 13, 2023 ... Nasa says its Perseverance rover has essen

Notice how the agents used at first the “Search” tool to look for recent information about the Mars rover and then used the “Summarizer” tool for writing a summary.

LangChain provides an expansive toolkit that integrates various functions to improve the functionality of conversational agents. Here are some examples:

1. SerpAPI: This tool is an interface for the SerpAPI search engine, allowing the agent to perform robust online searches to pull in relevant data for a conversation or task.
2. PythonREPLTool: This unique tool enables the writing and execution of Python code within an agent. This opens up a wide range of possibilities for advanced computations and interactions within the conversation.

If you wish to add more specialized capabilities to your LangChain conversational agent, the platform offers the flexibility to create custom tools. By following the general tool creation guidelines provided in the LangChain documentation, you can develop tools tailored to the specific needs of your application.